# Calculate Stats on Tuning Distributions

In [1]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import scikit_posthocs as posthoc
from statsmodels.stats.multitest import multipletests
from hmmlearn import hmm

sys.path.insert(0, os.path.abspath(r'C:/Users/mmccann/repos/bonhoeffer/prey_capture/'))
import functions_plotting as fp

In [66]:
def h5_to_dict(filepath):
    with pd.HDFStore(filepath, 'r') as store:
        keys = store.keys()
        data = {key[1:]: store[key] for key in keys}
    return data


def kruskal_w_dunn(data, nan_policy='omit', alpha=0.05):
    statistic, p_val = stats.kruskal(*data, nan_policy=nan_policy)
    dunn_test = posthoc.posthoc_dunn([*data], p_adjust='holm')

    # Return just the corrected pvals instead of a whole matrix
    if dunn_test.shape[-1] > 2:
        p_val_corrected = np.array([dunn_test.iloc[0, i] for i in range(1, dunn_test.shape[-1])] + [dunn_test.iloc[1, -1]])
    else:
        p_val_corrected = np.array([dunn_test.iloc[0, i] for i in range(1, dunn_test.shape[-1])])
    return p_val, p_val_corrected


def mannwhitneyu_w_bonferroni(data, nan_policy='omit', alpha=0.05):
    statistic, p_val = stats.mannwhitneyu(*data, nan_policy=nan_policy)
    reject, p_val_corrected, _, _ = multipletests(p_val, alpha=alpha, method='holm')
    return p_val, p_val_corrected


def wilcoxon_w_bonferroni(data, nan_policy='omit', alpha=0.05):
    statistic, p_val = stats.wilcoxon(*data, nan_policy=nan_policy)
    reject, p_val_corrected, _, _ = multipletests(p_val, alpha=alpha, method='holm')
    return p_val, p_val_corrected


def sample_df(data1, data2):
    df1 = data1.dropna()
    df2 = data2.dropna() 

    if len(df1) > len(df2):
        df1 = df1.sample(len(df2), replace=True)
    elif len(df2) > len(df1):
        df2 = df2.sample(len(df1), replace=True)
    else:
        pass

    return df1, df2


def bootstrap_tests(datasets, key, tuning_kind, method, n_iters=10000, alpha=0.05, bootstrap_all=False):
    data = [ds[key][tuning_kind].dropna() for ds in datasets]

    # First run the test on the raw data
    if 'wilcoxon' in method.__name__:
        raw_test = method(sample_df(*data), alpha=alpha)
    elif 'mannwhitneyu' in method.__name__:
        raw_test = method(data, alpha=alpha)
    else:
        raw_test = method(data, alpha=alpha)

    # Now run the bootstrapping
    p_vals = np.zeros(n_iters)
    p_vals_corrected = np.zeros((n_iters, len(data)))

    sample_size = len(max(data, key=len))

    # Run bootstrap
    for i in range(n_iters):

        if bootstrap_all:
            sample_data = [d.sample(sample_size, replace=True) for d in data]
            # sample_data1  = data2.sample(sample_size, replace=True)
        else:
            sample_data = [data[0]] + [d.sample(sample_size, replace=True) for d in data[1:]]
            
        p_val, p_val_corrected = method(sample_data, alpha=alpha)
        p_vals[i] = p_val
        p_vals_corrected[i, :] = p_val_corrected

    # 95% confidence interval of pvals
    ci = stats.t.interval(0.95, len(p_vals) - 1, loc=np.nanmean(p_vals), scale=stats.sem(p_vals, nan_policy='omit'))

    if method == kruskal_w_dunn:
        # Since this is for multiple groups, need to run this column-wise

        # Corrected 95% CIs
        ci_corrected = np.zeros((len(data), 2))
        for i in range(p_vals_corrected.shape[-1]):
            p_val_corr_col = p_vals_corrected[:, i]
            ci_col = stats.t.interval(0.95, len(p_val_corr_col) - 1, loc=np.nanmean(p_val_corr_col), scale=stats.sem(p_val_corr_col, nan_policy='omit'))
            ci_corrected[i, :] = ci_col
    else:
        ci_corrected = stats.t.interval(0.95, len(p_vals_corrected) - 1, loc=np.nanmean(p_vals_corrected), scale=stats.sem(p_vals_corrected, nan_policy='omit'))

    return raw_test, (p_vals, ci), (p_vals_corrected, ci_corrected)


def plot_bootstrap_distributions(data):
    # Plot the bootstrap distributions
    num_ds = len(data)
    cols = 3
    rows = num_ds//cols + 1
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 5*rows))

    for i, (ds, ax) in enumerate(zip(data, axes.ravel())):

        raw_test = ds[0]
        bootstrap_tests = ds[1]
        corrected_bootstrap_tests = ds[2]

        ax.hist(bootstrap_tests[0], bins=20)
        ax.axvline(np.percentile(bootstrap_tests[0], 95), 0, 50, color='r')
        ax.axvline(bootstrap_tests[1][0], 0, 50, color='g')
        ax.axvline(bootstrap_tests[1][1], 0, 50, color='g')
        ax.axvline(raw_test[0], 0, 100, color='k')


    return fig, axes

In [4]:
basedir = r"Z:\Prey_capture\WF_Figures"
still_or_all = 'full'
main_exp_path = os.path.join(basedir, still_or_all, r"multi_normal_ALL\stats.hdf5")
control_light_path  = os.path.join(basedir, still_or_all, r"control_normal_ALL\stats.hdf5")
control_dark_path = os.path.join(basedir, still_or_all, r"control_dark_ALL\stats.hdf5")
repeat_fixed_path = os.path.join(basedir, still_or_all, r"repeat_normal_VWheelWF\stats.hdf5")
repeat_free_path = os.path.join(basedir, still_or_all, r"repeat_normal_VTuningWF\stats.hdf5")
output_path = os.path.join(basedir, 'stats', f"{still_or_all}_tuning_stats.h5")

In [5]:
main_exp_data = h5_to_dict(main_exp_path)
control_light_data = h5_to_dict(control_light_path)
control_dark_data = h5_to_dict(control_dark_path)
repeat_fixed_data = h5_to_dict(repeat_fixed_path)
repeat_free_data = h5_to_dict(repeat_free_path)

In [6]:
method = kruskal_w_dunn
bootstrap_iters = 1000

# Kinematic Tuning

In [67]:
def kinem_stats(data, ds_name, alpha=0.05):
    cols = data[0][ds_name].columns.to_list()

    stats = {}
    for col in cols:
        stats[col] = bootstrap_tests(data, ds_name, col,   
                                      kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)

    return stats

In [88]:
var = 'mouse_heading'
print(free_kinem[var][0])
print(free_kinem[var][-1][-1])

(0.10221279416257911, array([0.32058756, 0.16056516, 0.12267395]))
[[1.21113810e-01 1.47050606e-01]
 [1.48939219e-02 2.14869225e-02]
 [5.59826499e-05 3.68494937e-04]]


In [ ]:
# Fixed data
fixed_kinem = kinem_stats([main_exp_data, control_light_data, control_dark_data], 'frac_kinem_resp_fixed')
kinem_tuning_fixed = pd.DataFrame(fixed_kinem).T
kinem_tuning_fixed.columns = ['main', 'control_light', 'control_dark']

# Save it
with pd.HDFStore(output_path, 'a') as store:
    store['kruskal_kinem_tuning_fixed'] = kinem_tuning_fixed


# Free data
free_kinem = kinem_stats([main_exp_data, control_light_data, control_dark_data], 'frac_kinem_resp_free')
kinem_tuning_free = pd.DataFrame(free_kinem).T
kinem_tuning_free.columns = ['main', 'control_light', 'control_dark']

# Save it
with pd.HDFStore(output_path, 'a') as store:
    store['kruskal_kinem_tuning_free'] = kinem_tuning_free


# Visual Tuning

## Frac Visual Tuning - Fixed to Free

### Kruskal-Wallis

In [ ]:
# Do a Kruskal-Wallis test with posthoc Dunn test for multiple comparisons

# Visual
exp_fixed_free_vis = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'].visual, 
                                              main_exp_data['frac_vis_resp_free'].visual))
print('Fixed vs Free Visual:', exp_fixed_free_vis)


# Orientation
exp_fixed_free_ori = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'].orientation, 
                                              main_exp_data['frac_vis_resp_free'].orientation))
print('Fixed vs Free Orientation:', exp_fixed_free_ori)


# Direction
exp_fixed_free_dir = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'].direction, 
                                              main_exp_data['frac_vis_resp_free'].direction))
print('Fixed vs Free Direction:', exp_fixed_free_dir)

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store['kruskal_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
    store['kruskal_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
    store['kruskal_exp_fixed_free_dir'] = pd.DataFrame(exp_fixed_free_dir, index=['p_val', 'p_val_dunn_posthoc'])

Fixed vs Free Visual: (0.6368039160463146, array([0.63680392]))
Fixed vs Free Orientation: (5.562268506118308e-13, array([5.56226851e-13]))
Fixed vs Free Direction: (0.039924934730025854, array([0.03992493]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\1807864428.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store['kruskal_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\1807864428.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store['kruskal_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\1807864428.py:24: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key

### Wilcoxon Rank Sum

In [ ]:
# Do a Wilcox Rank-Sum test with Bonferroni correction

# Visual
exp_fixed_free_vis = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].visual, 
                                                     main_exp_data['frac_vis_resp_free'].visual))
print('Fixed vs Free Visual:', exp_fixed_free_vis)


# Orientation
exp_fixed_free_ori = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].orientation, 
                                                     main_exp_data['frac_vis_resp_free'].orientation))
print('Fixed vs Free Orientation:', exp_fixed_free_ori)


# Direction
exp_fixed_free_dir = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].direction, 
                                                     main_exp_data['frac_vis_resp_free'].direction))
print('Fixed vs Free Direction:', exp_fixed_free_dir)

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store['wilcoxon_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
    store['wilcoxon_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
    store['wilcoxon_exp_fixed_free_dir'] = pd.DataFrame(exp_fixed_free_dir, index=['p_val', 'p_val_bonferroni'])

Fixed vs Free Visual: (0.5535240471550497, array([0.55352405]))
Fixed vs Free Orientation: (2.1622325721292424e-17, array([2.16223257e-17]))
Fixed vs Free Direction: (0.0035842744078428378, array([0.00358427]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\4171216171.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store['wilcoxon_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\4171216171.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store['wilcoxon_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\4171216171.py:24: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->

### Mann-Whitney U

In [ ]:
# Do a Mann-Whitney U test with Bonferroni correction

# Visual
exp_fixed_free_vis = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].visual, 
                                              main_exp_data['frac_vis_resp_free'].visual))
print('Fixed vs Free Visual:', exp_fixed_free_vis)

# Orientation
exp_fixed_free_ori = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].orientation, 
                                              main_exp_data['frac_vis_resp_free'].orientation))
print('Fixed vs Free Orientation:', exp_fixed_free_ori)

# Direction
exp_fixed_free_dir = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].direction, 
                                              main_exp_data['frac_vis_resp_free'].direction))
print('Fixed vs Free Direction:', exp_fixed_free_dir)

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store['mannwhitneyu_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
    store['mannwhitneyu_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
    store['mannwhitneyu_exp_fixed_free_dir'] = pd.DataFrame(exp_fixed_free_dir, index=['p_val', 'p_val_bonferroni'])

Fixed vs Free Visual: (0.6369949704544589, array([0.63699497]))
Fixed vs Free Orientation: (5.573587631945802e-13, array([5.57358763e-13]))
Fixed vs Free Direction: (0.03995130385570297, array([0.0399513]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\1262552217.py:20: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store['mannwhitneyu_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\1262552217.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store['mannwhitneyu_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\1262552217.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mix

## Frac Visual Tuning - Exp vs Controls

### Fixed Kruskal Wallis

In [ ]:
# Do a Kruskal-Wallis test with Dunn posthoc

# Visual
frac_fixed_vis = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                 'frac_vis_resp_fixed', 'visual',
                                 kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Visual 95% CI', frac_fixed_vis[1][1])
print('Fixed vs Controls Visual 95% CI - Corrected:', frac_fixed_vis[-1][1])
print('Fixed vs Controls Visual Raw:', frac_fixed_vis[0])

# Orientation
frac_fixed_ori = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                 'frac_vis_resp_fixed', 'orientation',
                                 kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Ori 95% CI', frac_fixed_ori[1][1])
print('Fixed vs Controls Ori 95% CI - Corrected:', frac_fixed_ori[-1][1])
print('Fixed vs Controls Ori Raw:', frac_fixed_ori[0])

# Direction
frac_fixed_dir = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                 'frac_vis_resp_fixed', 'direction',
                                 kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Dir 95% CI', frac_fixed_dir[1][1])
print('Fixed vs Controls Dir 95% CI - Corrected:', frac_fixed_dir[-1][1])
print('Fixed vs Controls Dir Raw:', frac_fixed_dir[0])

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store['kruskal_frac_fixed_vis'] = pd.DataFrame(list(frac_fixed_vis), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_frac_fixed_ori'] = pd.DataFrame(list(frac_fixed_ori), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_frac_fixed_dir'] = pd.DataFrame(list(frac_fixed_dir), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

Fixed vs Controls Visual 95% CI (-2.828481708825561e-29, 8.838416656002784e-29)
Fixed vs Controls Visual 95% CI - Corrected: [[-1.04640460e-22  4.60604228e-21]
 [-1.02163626e-24  5.56194443e-24]
 [ 4.36831988e-01  4.73458884e-01]]
Fixed vs Controls Visual Raw: (8.366268871528825e-15, array([1.57344464e-07, 9.83506935e-11, 6.07587104e-01]))
Fixed vs Controls Ori 95% CI (-1.0453309158973413e-64, 3.353553894210938e-64)
Fixed vs Controls Ori 95% CI - Corrected: [[-6.41434501e-51  3.00994560e-50]
 [-4.39958159e-50  1.15057650e-48]
 [ 4.85532980e-01  5.14752960e-01]]
Fixed vs Controls Ori Raw: (1.4468502067743607e-19, array([3.18220818e-11, 1.40078048e-12, 8.89164817e-01]))
Fixed vs Controls Dir 95% CI (-3.469602102578267e-36, 1.8372800574377145e-35)
Fixed vs Controls Dir 95% CI - Corrected: [[ 1.25010240e-34  6.64081190e-32]
 [-2.44738871e-22  3.05852556e-21]
 [ 9.00474963e-02  1.06765360e-01]]
Fixed vs Controls Dir Raw: (2.9913270159495296e-13, array([2.46097057e-08, 3.44884283e-08, 6.0669

C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\2937115539.py:29: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store['kruskal_frac_fixed_vis'] = pd.DataFrame(list(frac_fixed_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\2937115539.py:31: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store['kruskal_frac_fixed_ori'] = pd.DataFrame(list(frac_fixed_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\2937115539.py:33: PerformanceWarning: 
your performance may suffer as PyTables will pickle objec

In [ ]:
frac_fixed_fig = plot_bootstrap_distributions([frac_fixed_gen, 
                                               frac_fixed_vis, frac_fixed_vis_no_gen, 
                                               frac_fixed_ori, frac_fixed_ori_no_gen,
                                               frac_fixed_dir, frac_fixed_dir_no_gen])
plt.show()

### Free Kruskal Wallis

In [ ]:
# Do a Kruskal-Wallis test with Dunn posthoc

# Visual
frac_free_vis = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                  'frac_vis_resp_free', 'visual',
                                  kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Visual 95% CI', frac_free_vis[1][1])
print('Free vs Controls Visual 95% CI - Corrected:', frac_free_vis[-1][1])
print('Free vs Controls Visual Raw:', frac_free_vis[0])

# Orientation
frac_free_ori = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                  'frac_vis_resp_free', 'orientation',
                                  kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Ori 95% CI', frac_free_ori[1][1])
print('Free vs Controls Ori 95% CI - Corrected:', frac_free_ori[-1][1])
print('Free vs Controls Ori Raw:', frac_free_ori[0])

# Direction
frac_free_dir = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                  'frac_vis_resp_free', 'direction',
                                  kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Dir 95% CI', frac_free_dir[1][1])
print('Free vs Controls Dir 95% CI - Corrected:', frac_free_dir[-1][1])
print('Free vs Controls Dir Raw:', frac_free_dir[0])


# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store['kruskal_frac_free_vis'] = pd.DataFrame(list(frac_free_vis), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_frac_free_ori'] = pd.DataFrame(list(frac_free_ori), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_frac_free_dir'] = pd.DataFrame(list(frac_free_dir), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

Free vs Controls Visual 95% CI (-3.378824251318259e-25, 4.496966286521529e-24)
Free vs Controls Visual 95% CI - Corrected: [[ 3.64541627e-18  4.85787939e-17]
 [-3.03196346e-20  1.68238257e-19]
 [ 3.48658860e-01  3.83911276e-01]]
Free vs Controls Visual Raw: (2.256976482633488e-12, array([7.64863616e-07, 1.22705793e-08, 8.38302451e-01]))
Free vs Controls Ori 95% CI (-4.4520023719633245e-17, 4.718578427914253e-16)
Free vs Controls Ori 95% CI - Corrected: [[ 3.62446534e-08  1.72921695e-07]
 [-1.76723247e-16  1.42809905e-15]
 [ 7.21194156e-03  1.08306496e-02]]
Free vs Controls Ori Raw: (1.215172622536174e-07, array([1.63979021e-03, 2.42977059e-06, 3.70787550e-01]))
Free vs Controls Dir 95% CI (-1.5006553818339583e-20, 7.513208770557706e-20)
Free vs Controls Dir 95% CI - Corrected: [[ 2.58086836e-06  9.54486531e-06]
 [-5.10161402e-21  2.56370613e-20]
 [ 2.39666539e-06  5.88676774e-05]]
Free vs Controls Dir Raw: (6.262589083172232e-09, array([4.42860952e-03, 4.12671462e-08, 9.76719490e-02]))

C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\4050277121.py:30: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store['kruskal_frac_free_vis'] = pd.DataFrame(list(frac_free_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\4050277121.py:32: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store['kruskal_frac_free_ori'] = pd.DataFrame(list(frac_free_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\4050277121.py:34: PerformanceWarning: 
your performance may suffer as PyTables will pickle object ty

In [ ]:
frac_free_fig = plot_bootstrap_distributions([frac_free_gen, 
                                              frac_free_vis, frac_free_vis_no_gen, 
                                              frac_free_ori, frac_free_ori_no_gen,
                                              frac_free_dir, frac_free_dir_no_gen])
plt.show()

## Count Vis Tuning - Fixed vs Free

### Kruskal-Wallis

In [ ]:
# Do a Kruskal-Wallis test with posthoc Dunn test for multiple comparisons

# Visual
count_exp_fixed_free_vis = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'].visual, 
                                              main_exp_data['count_vis_resp_free'].visual))
print('Fixed vs Free Visual:', exp_fixed_free_vis)

# Orientation
count_exp_fixed_free_ori = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'].orientation, 
                                              main_exp_data['count_vis_resp_free'].orientation))
print('Fixed vs Free Orientation:', exp_fixed_free_ori)


# Direction
count_exp_fixed_free_dir = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'].direction, 
                                              main_exp_data['count_vis_resp_free'].direction))
print('Fixed vs Free Direction:', exp_fixed_free_dir)

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store['kruskal_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
    store['kruskal_exp_fixed_free_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
    store['kruskal_exp_fixed_free_dir_count'] = pd.DataFrame(count_exp_fixed_free_dir, index=['p_val', 'p_val_dunn_posthoc'])

Fixed vs Free Visual: (0.6369949704544589, array([0.63699497]))
Fixed vs Free Orientation: (5.573587631945802e-13, array([5.57358763e-13]))
Fixed vs Free Direction: (0.03995130385570297, array([0.0399513]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\557711235.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store['kruskal_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\557711235.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store['kruskal_exp_fixed_free_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\557711235.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inf

### Wilcoxon Rank Sum

In [ ]:
# Do a Wilcox Rank-Sum test with Bonferroni correction

# Visual
count_exp_fixed_free_vis = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].visual, 
                                              main_exp_data['count_vis_resp_free'].visual))
print('Fixed vs Free Visual:', exp_fixed_free_vis)

# Orientation
count_exp_fixed_free_ori = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].orientation, 
                                              main_exp_data['count_vis_resp_free'].orientation))
print('Fixed vs Free Orientation:', exp_fixed_free_ori)

# Direction
count_exp_fixed_free_dir = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].direction, 
                                              main_exp_data['count_vis_resp_free'].direction))
print('Fixed vs Free Direction:', exp_fixed_free_dir)

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store['wilcoxon_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
    store['wilcoxon_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
    store['wilcoxon_exp_fixed_free_dir_count'] = pd.DataFrame(count_exp_fixed_free_dir, index=['p_val', 'p_val_dunn_posthoc'])

Fixed vs Free Visual: (0.6369949704544589, array([0.63699497]))
Fixed vs Free Orientation: (5.573587631945802e-13, array([5.57358763e-13]))
Fixed vs Free Direction: (0.03995130385570297, array([0.0399513]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\2056095805.py:20: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store['wilcoxon_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\2056095805.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store['wilcoxon_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\2056095805.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-type

### Mann-Whitney U

In [ ]:
# Do a Mann-Whitney U test with Bonferroni correction

# Visual
count_exp_fixed_free_vis = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].visual, 
                                              main_exp_data['count_vis_resp_free'].visual))
print('Fixed vs Free Visual:', exp_fixed_free_vis)

# Orientation
count_exp_fixed_free_ori = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].orientation, 
                                              main_exp_data['count_vis_resp_free'].orientation))
print('Fixed vs Free Orientation:', exp_fixed_free_ori)

# Direction
count_exp_fixed_free_dir = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].direction, 
                                              main_exp_data['count_vis_resp_free'].direction))
print('Fixed vs Free Direction:', exp_fixed_free_dir)

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store['mannwhitneyu_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
    store['mannwhitneyu_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
    store['mannwhitneyu_exp_fixed_free_dir_count'] = pd.DataFrame(count_exp_fixed_free_dir, index=['p_val', 'p_val_dunn_posthoc'])

Fixed vs Free Visual: (0.6369949704544589, array([0.63699497]))
Fixed vs Free Orientation: (5.573587631945802e-13, array([5.57358763e-13]))
Fixed vs Free Direction: (0.03995130385570297, array([0.0399513]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\492689366.py:20: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store['mannwhitneyu_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\492689366.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store['mannwhitneyu_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\492689366.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c

## Count Vis Tuning - Exp vs Controls

### Fixed Kruskal Wallis

In [ ]:
# Do a Kruskal-Wallis test with Dunn posthoc


# Visual
count_fixed_vis = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                 'count_vis_resp_fixed', 'visual',   
                                 kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Visual 95% CI:', count_fixed_vis[1][1])
print('Fixed vs Controls Visual Raw:', count_fixed_vis[0])

# Orientation
count_fixed_ori = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                 'count_vis_resp_fixed', 'orientation',
                                 kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Orientation 95% CI:', count_fixed_ori[1][1])
print('Fixed vs Controls Orientation Raw:', count_fixed_ori[0])

# Direction
count_fixed_dir = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                 'count_vis_resp_fixed', 'direction',
                                 kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Direction 95% CI:', count_fixed_dir[1][1])
print('Fixed vs Controls Direction Raw:', count_fixed_dir[0])

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store['kruskal_count_fixed_vis'] = pd.DataFrame(list(count_fixed_vis), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_count_fixed_ori'] = pd.DataFrame(list(count_fixed_ori), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_count_fixed_dir'] = pd.DataFrame(list(count_fixed_dir), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

Fixed vs Controls Visual 95% CI: (1.1898280240107973e-07, 4.278181580262835e-07)
Fixed vs Controls Visual Raw: (0.000268196058097534, array([0.00634131, 0.00634131, 0.78859931]))
Fixed vs Controls Orientation 95% CI: (-1.0588450281132102e-23, 1.787913192627451e-22)
Fixed vs Controls Orientation Raw: (5.443136292223412e-08, array([5.21956459e-05, 2.30637753e-05, 9.07642115e-01]))
Fixed vs Controls Direction 95% CI: (-1.7748405866532095e-10, 6.225623259710055e-10)
Fixed vs Controls Direction Raw: (1.6802916286478037e-05, array([0.00106744, 0.00106744, 0.73690399]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\1615279377.py:27: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store['kruskal_count_fixed_vis'] = pd.DataFrame(list(count_fixed_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\1615279377.py:29: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store['kruskal_count_fixed_ori'] = pd.DataFrame(list(count_fixed_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\1615279377.py:31: PerformanceWarning: 
your performance may suffer as PyTables will pickle o

### Free Kruskal Wallis

In [ ]:
# Do a Kruskal-Wallis test with Dunn posthoc

# Visual
count_free_vis = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                 'count_vis_resp_free', 'visual',   
                                 kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Visual 95% CI:', count_free_vis[1][1])
print('Free vs Controls Visual Raw:', count_free_vis[0])

# Orientation
count_free_ori = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                 'count_vis_resp_free', 'orientation',
                                 kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Orientation 95% CI:', count_free_ori[1][1])
print('Free vs Controls Orientation Raw:', count_free_ori[0])

# Direction
count_free_dir = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                 'count_vis_resp_free', 'direction',
                                 kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Direction 95% CI:', count_free_dir[1][1])
print('Free vs Controls Direction Raw:', count_free_dir[0])

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store['kruskal_count_free_vis'] = pd.DataFrame(list(count_free_vis), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_count_free_ori'] = pd.DataFrame(list(count_free_ori), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_count_free_dir'] = pd.DataFrame(list(count_free_dir), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

Free vs Controls Visual 95% CI: (-1.5467932053140595e-06, 1.1222112647587745e-05)
Free vs Controls Visual Raw: (0.0003101948186057097, array([0.01768822, 0.00235345, 0.73407518]))
Free vs Controls Orientation 95% CI: (1.356590257564576e-07, 8.352331534934261e-07)
Free vs Controls Orientation Raw: (0.001265652736278355, array([0.06810468, 0.00383673, 0.5248231 ]))
Free vs Controls Direction 95% CI: (1.7544569059523954e-09, 2.5175397273982187e-08)
Free vs Controls Direction Raw: (0.0002634136057045376, array([0.08240132, 0.00055088, 0.28396112]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\1888198755.py:26: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store['kruskal_count_free_vis'] = pd.DataFrame(list(count_free_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\1888198755.py:28: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store['kruskal_count_free_ori'] = pd.DataFrame(list(count_free_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_21936\1888198755.py:30: PerformanceWarning: 
your performance may suffer as PyTables will pickle objec

# Running Modulated

In [52]:
list(main_exp_data.keys())

['angle_residual_rmse_tuned_all_matches_all_vis_resp',
 'angle_residual_rmse_tuned_all_matches_both_vis_resp',
 'angle_residual_rmse_tuned_curated_matches_all_vis_resp',
 'angle_residual_rmse_tuned_curated_matches_both_vis_resp',
 'angle_residual_rmse_vis_resp_all_matches_all_vis_resp',
 'angle_residual_rmse_vis_resp_all_matches_both_vis_resp',
 'angle_residual_rmse_vis_resp_curated_matches_all_vis_resp',
 'angle_residual_rmse_vis_resp_curated_matches_both_vis_resp',
 'count_vis_resp_fixed',
 'count_vis_resp_free',
 'dsi_residuals_tuned_all_matches_all_vis_resp',
 'dsi_residuals_tuned_all_matches_both_vis_resp',
 'dsi_residuals_tuned_curated_matches_all_vis_resp',
 'dsi_residuals_tuned_curated_matches_both_vis_resp',
 'dsi_residuals_vis_resp_all_matches_all_vis_resp',
 'dsi_residuals_vis_resp_all_matches_both_vis_resp',
 'dsi_residuals_vis_resp_curated_matches_all_vis_resp',
 'dsi_residuals_vis_resp_curated_matches_both_vis_resp',
 'dsi_shifts_tuned_all_matches_all_vis_resp',
 'dsi_shi